In [1]:
import pe_ode as pe
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp, odeint

# Test of Parameter Estimation method for ODE model
## Model
A homogeneous gas phase reaction
$$2NO+O_2\leftrightarrow2NO_2$$
is described by the following equation
$$\frac{dy}{dt}=k_1(126.2-y)(91.9-y)^2-k_2y^2;~~y(0)=0$$

In [3]:
def homo_gas(y,k):
    dydt = k[0]*(126.2-y)*(91.9-y)**2-k[1]*y**2
    return dydt

Pyrolytic dehydrogenation of dehzene to diphenyl and triphenyl
$$2C_2H_6\longleftrightarrow C_{12}H_{10}+H_2$$
$$C_6H_6+C_{12}H_{10}\longleftrightarrow C_{10}H_{14}+H_2$$
with differential equations model
\begin{align*}
\frac{dy_1}{dt}=&-r_1-r_2;~~y_1(0)=1\\
\frac{dy_2}{dt}=&\frac{r_1}{2}-r_2;~~~~y_2(0)=0\\
r_1=&k_1\left(y_1^2-y_2\frac{2-2y_1-y_2}{3K_1}\right)\\
r_2=&k_2\left(y_1y_2-\frac{(1-y_1-2y_2)(2-2y_1-y_2)}{9K_2}\right)
\end{align*}
with $K_1=0.242$ and $K_2=0.428$

In [4]:
def dehydro_ben(y,k):
    K1 = 0.242
    K2 = 0.428
    r1 = k[0]*(y[0]**2-y[1]*(2-2*y[0]-y[1])/(3*K1))
    r2 = k[1]*(y[0]*y[1]-(1-y[0]-2*y[1])*(2-2*y[0]-y[1])/(9*K2))
    dydt = np.empty(2)
    dydt[0] = -r1-r2
    dydt[1] = r1/2-r2
    return dydt

## Jacobian
Differential equation $\frac{d\mathbf{J}}{dt}$ with $\mathbf{J}=\frac{\partial\mathbf{y}}{\partial\mathbf{k}}$ is
$$\frac{d\mathbf{J}}{dt}=\frac{\partial\mathbf{f}}{\partial\mathbf{y}}\mathbf{J}+\frac{\partial\mathbf{f}}{\partial\mathbf{k}}$$
For this model
$$\frac{\partial f}{\partial y}=-k_1(91.9-y)^2-2k_1(126.2-y)(91.9-y)-2k_2y$$
$$\frac{\partial f}{\partial k_1}=(126.2-y)(91.9-y)^2$$
$$\frac{\partial f}{\partial k_2}=-y^2$$

In [5]:
def homo_gas_dfdy(y,k):
    dfdy = -k[0]*(91.9-y)**2-2*k[0]*(126.2-y)*(91.9-y)-2*k[1]*y
    return dfdy
def homo_gas_dfdk(y,k):
    dfdk = np.empty(2)
    dfdk[0] = (126.2-y)*(91.9-y)**2
    dfdk[1] = -y**2
    return dfdk

For the dehydrogenation model
\begin{align*}
r_1=&2y_1+\frac{2y_2}{3K_1}\\
r_2=&\frac{2y_1+2y_2-2}{3K_1}\\
r_3=&y_2-\frac{4y_1+5y_2-4}{9K_2}\\
r_4=&y_1-\frac{5y_1+4y_2-5}{9K_2}\\\\
\frac{\partial f_1}{\partial y_1}=&-k_1r_1-k_2r_3,\hspace{1cm}\frac{\partial f_1}{\partial y_2}=-k_1r_2-k_2r_4\\
\frac{\partial f_2}{\partial y_1}=&\frac{k_1}{2}r_1-k_2r_3,\hspace{1.3cm}\frac{\partial f_2}{\partial y_2}=\frac{k_1}{2}r_2-k_2r_4\\\\
\frac{\partial f_1}{\partial k_1}=&-\left(y_1^2+\frac{y_2^2+2y_1y_2-2y_2}{3K_1}\right)\\
\frac{\partial f_1}{\partial k_2}=&-\left(y_1y_2-\frac{2y_1^2-4y_1+5y_1y_2-5y_2+2y_2^2+2}{9K_2}\right)\\
\frac{\partial f_2}{\partial k_1}=&-\frac{1}{2}\frac{\partial f_1}{\partial k_1}\\
\frac{\partial f_2}{\partial k_2}=&\frac{\partial f_1}{\partial k_2}
\end{align*}

In [6]:
def dehydro_dfdy(y,k):
    K1 = 0.242
    K2 = 0.428
    r1 = 2*y[0]+2*y[1]/(3*K1)
    r2 = 2*(y[0]+y[1]-1)/(3*K1)
    r3 = y[1]-(4*y[0]+5*y[1]-4)/(9*K2)
    r4 = y[0]-(5*y[0]+4*y[1]-5)/(9*K2)
    dfdy = np.empty((2,2))
    dfdy[0,0] = -k[0]*r1-k[1]*r3
    dfdy[0,1] = -k[0]*r2-k[1]*r4
    dfdy[1,0] = k[0]*r1/2-k[1]*r3
    dfdy[1,1] = k[0]*r2/2-k[1]*r4
    return dfdy
def dehydro_dfdk(y,k):
    K1 = 0.242
    K2 = 0.428
    dfdk = np.empty((2,2))
    dfdk[0,0] = -(y[0]**2+(y[1]**2+2*y[0]*y[1]-2*y[1])/(3*K1))
    dfdk[0,1] = -(y[0]*y[1]-(2*y[0]**2-4*y[0]+5*y[0]*y[1]-5*y[1]+2*y[1]**2+2)/(9*K2))
    dfdk[1,0] = -dfdk[0,0]/2
    dfdk[1,1] = dfdk[0,1]
    return dfdk

## Comparison with numerical differentiation
Numerical differentiation by GN_ode functions and analytic results given above will be compared. Homogeneous gas model has 15 measurements and the parameter estimations are $k_1=0.4577\times10^{-5}$ and $k_2=0.2796\times10^{-3}$

In [7]:
yhat_homo=np.array([[0,1.4,6.3,10.5,14.2,17.6,21.4,23.0,27.0,30.5,34.4,38.8,41.6,43.5,45.3]])
t_homo = np.array([0,1,2,3,4,5,6,7,9,11,14,19,24,29,39])
k_homo = np.array([0.4577e-5,0.2796e-3])

In [8]:
kkk = np.array([10,10])
dfdy_anal = np.empty(15)
dfdy_num = np.empty(15)
dfdk_anal = np.empty((2,15))
dfdk_num = np.empty((2,15))
y_temp = gn.state_only_int(homo_gas,np.array([0]),kkk,t_homo)
y_kkk = y_temp[0]
for i in range(15):
    dfdy_anal[i] = homo_gas_dfdy(y_kkk[:,i],kkk)
    dfdy_num[i] = gn.dfdy_ode(homo_gas,y_kkk[:,i],kkk,1)
    dfdk_anal[:,i] = homo_gas_dfdk(y_kkk[:,i],kkk)
    dfdk_num[:,i] = gn.dfdk_ode(homo_gas,y_kkk[:,i],kkk,1,2)
dfdy_diff = dfdy_anal-dfdy_num
dfdk_diff = dfdk_anal-dfdk_num
rel_dfdy = np.max(np.abs(dfdy_diff))/np.linalg.norm(dfdy_anal,np.inf)
rel_dfdk = np.max(np.abs(dfdk_diff))/np.linalg.norm(dfdk_anal,np.inf)
print('dfdy difference {0:1.2e} (inf norm)'.format(rel_dfdy))    
print('dfdk difference {0:1.2e} (inf norm)'.format(rel_dfdk))

dfdy difference 8.12e-07 (inf norm)
dfdk difference 2.31e-08 (inf norm)


Dehydrogenation model has eight measurements and the parameter estimations are $k_1=354.61$ and $k_2=400.23$

In [9]:
t_deh = np.array([0,5.63,11.32,16.97,22.62,34.00,39.70,45.20,169.7],dtype='f')*1e-4
yhat_deh = np.array([[1,0.828,0.704,0.622,0.565,0.499,0.482,0.470,0.443],
                   [0,0.0737,0.1130,0.1322,0.1400,0.1468,0.1477,0.1477,0.1476]],dtype='f')
y0_deh = yhat_deh[:,0]
k_deh = np.array([354.61,400.23])

In [10]:
k1000 = np.array([1000,1000],dtype='float64')
N_deh = np.size(t_deh)
dfdy_deh_ana = np.empty((N_deh,2,2))
dfdy_deh_num = np.empty((N_deh,2,2))
dfdk_deh_ana = np.empty((N_deh,2,2))
dfdk_deh_num = np.empty((N_deh,2,2))
y1000,fasu = gn.state_only_int(dehydro_ben,y0_deh,k1000,t_deh)
for i in range(N_deh):
    dfdy_deh_ana[i] = dehydro_dfdy(y1000[:,i],k1000)
    dfdy_deh_num[i] = gn.dfdy_ode(dehydro_ben,y1000[:,i],k1000,2)
    dfdk_deh_ana[i] = dehydro_dfdk(y1000[:,i],k1000)
    dfdk_deh_num[i] = gn.dfdk_ode(dehydro_ben,y1000[:,i],k1000,2,2)
dfdy_diff = dfdy_deh_ana-dfdy_deh_num
dfdk_diff = dfdk_deh_ana-dfdk_deh_num
rel_dfdy = np.linalg.norm(dfdy_diff)/np.linalg.norm(dfdy_deh_ana)
rel_dfdk = np.linalg.norm(dfdk_diff)/np.linalg.norm(dfdk_deh_ana)
print('dfdy difference {0:1.2e}'.format(rel_dfdy))
print('dfdk difference {0:1.2e}'.format(rel_dfdk))

dfdy difference 2.69e-09
dfdk difference 2.87e-06


## Test of $\varphi(\mathbf{z})$
With an artificial initial condition $y_0=10$, $\mathbf{J}=\begin{bmatrix}1000&2000\end{bmatrix}$
$$\frac{dy}{dt}=4.577\times10^{-6}(126.2-10)(91.9-10)^2-2.796\times10^{-4}\times10^2=3.5395$$
$$\frac{\partial f}{\partial y}=-4.577\times10^{-6}(91.9-10)^2-2\times4.577\times10^{-6}(126.2-10)(91.9-10)-2\times2.796\times10^{-4}\times10=-0.1234$$
$$\frac{\partial f}{\partial k_1}=(126.2-10)(91.9-10)^2=779424$$
$$\frac{\partial f}{\partial k_2}=-10^2=-100$$
The differential equation for Jacobian is
$$\frac{d\mathbf{J}}{dt}=-0.1234\times\begin{bmatrix}1000 & 2000\end{bmatrix}+\begin{bmatrix}779424&-100\end{bmatrix}=\begin{bmatrix}779301&-346.8\end{bmatrix}$$

In [11]:
y_temp = 10
dfdy_homo_ana = homo_gas_dfdy(y_temp,k_homo)
dfdk_homo_ana = homo_gas_dfdk(10,k_homo)
J_temp = np.array([1000,2000])
dJdt_homo_ana = dfdy_homo_ana*J_temp+dfdk_homo_ana
print('Analytic dJdt is ',dJdt_homo_ana)
z = np.empty(3)
z[0] = y_temp
z[1:] = J_temp
dJdt_homo_num = gn.phi_z(homo_gas,z,k_homo,1,2)
print('Numerical dJdt is ',dJdt_homo_num[1:])

Analytic dJdt is  [ 7.79300873e+05 -3.46818670e+02]
Numerical dJdt is  [ 7.79300873e+05 -3.46818788e+02]


Artificial initial condition $y0=[5~~10]$ and $\mathbf{J}=\begin{bmatrix}
200&300\\
150&50\end{bmatrix}$
$$\frac{d\mathbf{J}}{dt}=\frac{\partial\mathbf{f}}{\partial\mathbf{y}}\mathbf{J}+\frac{\partial\mathbf{f}}{\partial\mathbf{k}}$$

In [12]:
y_deh_temp0 = np.array([5,10])
J_deh_temp = np.array([[200,300],[150,50]])
dfdy_deh_temp = dehydro_dfdy(y_deh_temp0,k_deh)
dfdk_deh_temp = dehydro_dfdk(y_deh_temp0,k_deh)
dJdt_deh_ana = np.matmul(dfdy_deh_temp,J_deh_temp)+dfdk_deh_temp
z_deh = np.empty(6)
z_deh[0:2] = y_deh_temp0
z_deh[2:] = J_deh_temp.transpose().flatten()
dzdt_deh = gn.phi_z(dehydro_ben,z_deh,k_deh,2,2)
dJdt_deh_num = dzdt_deh[2:].reshape(2,2).transpose()
print('Analytic dJdt is \n',dJdt_deh_ana)
print('Numerical dJdt is \n',dJdt_deh_num)

Analytic dJdt is 
 [[-3508740.85315775 -3610034.82730877]
 [ 3563353.91956953  3407433.3131871 ]]
Numerical dJdt is 
 [[-3508741.33291654 -3610035.15135963]
 [ 3563354.37318194  3407433.77309991]]


## Integration with estimated parameters

In [13]:
y0 = yhat_homo[:,0]
res_homo = gn.state_jacob_int(homo_gas,y0,k_homo,t_homo)

In [14]:
print(res_homo[0])
print(res_homo[1])
print(res_homo[2])

[[ 0.          4.54927429  8.52410322 12.02631169 15.13385962 17.90748572
  20.39634002 22.63844317 26.50804658 29.71156267 33.56817887 38.19757805
  41.3294756  43.48731834 46.05863949]]
[array([[0., 0.]]), array([[ 9.27370654e+05, -6.89875129e+00]]), array([[ 1.63014749e+06, -4.84419743e+01]]), array([[ 2.16757281e+06, -1.44611048e+02]]), array([[ 2.58077504e+06, -3.05211321e+02]]), array([[ 2.89900622e+06, -5.33798827e+02]]), array([[ 3.14403358e+06, -8.30288775e+02]]), array([[ 3.33115141e+06, -1.19188702e+03]]), array([[ 3.57842540e+06, -2.09397023e+03]]), array([[ 3.70681368e+06, -3.20044231e+03]]), array([[3763593.48611649,   -5150.32635678]]), array([[3669696.49666203,   -8853.71520985]]), array([[3482142.95125699,  -12718.05799824]]), array([[3274617.8936603 ,  -16431.03234524]]), array([[2905273.18131808,  -22776.51257757]])]
True


In [15]:
res_deh = gn.state_jacob_int(dehydro_ben,y0_deh,k_deh,t_deh)
print(res_deh[0])
print(res_deh[1])
print(res_deh[2])

[[1.         0.82832973 0.70541334 0.62146551 0.56431598 0.49897287
  0.4811068  0.46933922 0.44329947]
 [0.         0.07378088 0.11219659 0.13131036 0.14070327 0.14727456
  0.14814051 0.14843773 0.14773843]]
[array([[0., 0.],
       [0., 0.]]), array([[-3.90267081e-04, -1.62685326e-05],
       [ 1.69437807e-04, -1.93878548e-05]]), array([[-5.26574250e-04, -3.82839771e-05],
       [ 2.08403141e-04, -5.19365181e-05]]), array([[-5.25552588e-04, -5.14564067e-05],
       [ 1.97280549e-04, -7.60591874e-05]]), array([[-4.65126820e-04, -5.58872386e-05],
       [ 1.70508583e-04, -8.72084943e-05]]), array([[-3.07581678e-04, -4.93449214e-05],
       [ 1.13525940e-04, -8.08521169e-05]]), array([[-2.38931834e-04, -4.27692119e-05],
       [ 8.98380206e-05, -7.06632214e-05]]), array([[-1.84061115e-04, -3.60878218e-05],
       [ 7.07467410e-05, -5.98401691e-05]]), array([[-1.30737774e-07, -5.85794893e-08],
       [ 7.75995370e-08, -1.26581694e-07]])]
True


## Calculation of $\Delta k$ and $\chi^2$ with guessed parameters
$$k_{guess}=\begin{bmatrix}1\times10^{-6}&1\times10^{-6}\end{bmatrix}$$

In [16]:
k_guess = np.array([1e-6,1e-6])
Y_guess,J_guess,fs = gn.state_jacob_int(homo_gas,y0,k_guess,t_homo)
Q = np.eye(1)
dk_homo = gn.delta_k(J_guess,Q,yhat_homo,Y_guess,2,15)
chi_homo = gn.chi_squared(yhat_homo,Y_guess,Q,15)
print('delta k is ',dk_homo)
print('chi squared is {0:.4e}'.format(chi_homo))

delta k is  [2.38247336e-06 2.58667199e-03]
chi squared is 4.0117e+03


$$k_\text{guess}=[10000~~10000]$$

In [17]:
k_deh_guess = np.array([10000,10000],dtype='f')
Y_deh_guess,J_deh_guess,fs = gn.state_jacob_int(dehydro_ben,y0_deh,k_deh_guess,t_deh)
Q_deh = np.eye(2)
dk_deh = gn.delta_k(J_deh_guess,Q_deh,yhat_deh,Y_deh_guess,2,N_deh)
chi_deh = gn.chi_squared(yhat_deh,Y_deh_guess,Q_deh,N_deh)
print('delta k is ',dk_deh)
print('chi squared is ',chi_deh)
y_try,fs = gn.state_only_int(dehydro_ben,y0_deh,k_deh_guess,t_deh)

delta k is  [-49747007.5221476   -4368270.37717827]
chi squared is  0.27500952193073125


## Running a step for $k^{(2)}$ with bisection rule

In [18]:
bisec_homo = gn.bisect(homo_gas,yhat_homo,Q,k_guess,t_homo,100)

In [19]:
Yhomo_cur = bisec_homo[0]
Yhomo_nxt = bisec_homo[1]
Jhomo_cur = bisec_homo[2]
delta_k_homo = bisec_homo[3]
mu_homo_acpt = bisec_homo[4]

In [20]:
mu_homo_opt = gn.optimal_step_size(Yhomo_cur,Yhomo_nxt,yhat_homo,Jhomo_cur,delta_k_homo,mu_homo_acpt,Q,1,15)
print(mu_homo_opt)

[0.65383084+1.01516079j 0.65383084-1.01516079j 0.75468282+0.j        ]


In [21]:
y_acpt,fos = gn.state_only_int(homo_gas,y0,k_guess+delta_k_homo,t_homo)
y_opt,fos = gn.state_only_int(homo_gas,y0,k_guess+0.754682*delta_k_homo,t_homo)
chi_homo_acpt = gn.chi_squared(yhat_homo,y_acpt,Q,15)
chi_homo_opt = gn.chi_squared(yhat_homo,y_opt,Q,15)
print(chi_homo_acpt)
print(chi_homo_opt)

1739.9030984445694
1853.773356872011


In [22]:
bisection = gn.bisect(dehydro_ben,yhat_deh,Q_deh,k_deh_guess,t_deh,100)

In [23]:
Y_cur = bisection[0]
Y_nxt = bisection[1]
J_cur = bisection[2]
delta_k = bisection[3]
mu_acpt = bisection[4]

In [24]:
mu_opt = gn.optimal_step_size(Y_cur,Y_nxt,yhat_deh,J_cur,delta_k,mu_acpt,Q_deh,2,9)

In [25]:
k2_deh = k_deh_guess+mu_acpt*delta_k
print(k2_deh)
print(mu_opt)

[3927.36724583 9466.76386997]
[-8.83184480e-04  8.82442288e-04 -4.18239817e-07]


In [26]:
a = np.empty((3,2))
a

array([[-8.83184480e-04,  0.00000000e+00],
       [ 8.82442288e-04,  0.00000000e+00],
       [-4.18239817e-07,  0.00000000e+00]])